-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

-sandbox

<img src="http://spark-mooc.github.io/web-assets/images/cs110x/movie-camera.png" style="float:right; height: 200px; margin: 10px; border: 1px solid #ddd; border-radius: 15px 15px 15px 15px; padding: 10px"/>

# Predicting Movie Ratings

One of the most common uses of big data is to predict what users want.  This allows Google to show you relevant ads, Amazon to recommend relevant products, and Netflix to recommend movies that you might like.  This lab will demonstrate how we can use Apache Spark to recommend movies to a user.  We will start with some basic techniques, and then use the SparkML library's Alternating Least Squares method to make more sophisticated predictions. Here are the SparkML <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html" target="_blank">Python docs</a>.

For this lab, we will use 1 million movie ratings from the <a href="http://grouplens.org/datasets/movielens/" target="_blank">MovieLens stable benchmark rating dataset</a>. 

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - **Exploring the dataset and build a baseline model**
 - **Build a Collaborative Filtering model**
 - **Make customized movie predictions for yourself**

### Motivation: Want to win $1,000,000?

All you needed to do was improve Netflix’s movie recommendation system by 10% in 2008. This competition is known as the <a href="https://en.wikipedia.org/wiki/Netflix_Prize" target="_blank">Netflix Prize</a>. 

Good recommendations are vital to sites such as Netflix, where 75 percent of what consumers watch come from movie recommendations.

So, how do we create recommendations and evaluate their relevance?

In [0]:
%run "../Includes/Classroom-Setup"

Defining courseware-specific utility methods...

The source for this dataset is
wasbs://courseware@dbacademy.blob.core.windows.net/scalable-machine-learning-with-apache-spark/v01/

Your dataset directory is
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/machine_learning/datasets

Skipping install of existing dataset.

## Part 1: Exploring our Dataset

First, let's take a look at the directory containing our files.

In [0]:
files = dbutils.fs.ls(f"{datasets_dir}/movielens")
display(files)

path,name,size,modificationTime
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/machine_learning/datasets/movielens/movies.parquet/,movies.parquet/,0,1659952642000
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/machine_learning/datasets/movielens/ratings.parquet/,ratings.parquet/,0,1659952654000


### Load and Cache

We're going to be accessing this data a lot. 

Rather than reading it from source over and over again, we'll cache both the movies DataFrame and the ratings DataFrame into the executor's memory.

In [0]:
movies_df = spark.read.parquet(f"{datasets_dir}/movielens/movies.parquet/").cache()
ratings_df = spark.read.parquet(f"{datasets_dir}/movielens/ratings.parquet/").cache()

ratings_count = ratings_df.count()
movies_count = movies_df.count()

print(f"There are {ratings_count} ratings and {movies_count} movies in the datasets")

There are 1073207 ratings and 27278 movies in the datasets

Let's take a quick look at some of the data in the two DataFrames.

In [0]:
display(movies_df)

ID,title
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)
6,Heat (1995)
7,Sabrina (1995)
8,Tom and Huck (1995)
9,Sudden Death (1995)
10,GoldenEye (1995)


In [0]:
display(ratings_df)

userId,movieId,rating
19,1,5.0
19,3,4.0
19,6,5.0
19,7,5.0
19,14,4.0
19,17,4.0
19,25,3.0
19,32,2.0
19,55,3.0
19,61,4.0


## **Part 2: Collaborative Filtering**

### (2a) Creating a Training Set

Before we jump into using machine learning, we need to break up the **`ratings_df`** dataset into two DataFrames:
* A training set, which we will use to train models
* A test set, which we will use for our experiments

To randomly split the dataset into the multiple groups, we can use the <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.randomSplit.html?highlight=randomsplit#pyspark.sql.DataFrame.randomSplit" target="_blank">randomSplit()</a> transformation. **`randomSplit()`** takes a set of splits and a seed and returns multiple DataFrames. Use the seed given below.

In [0]:
# TODO
# We'll hold out 80% for training and leave 20% for testing 
seed = 42
train_df, test_df = ratings_df.randomSplit([0.8,0.2],seed=seed)

print(f"Training: {train_df.count()}, test: {test_df.count()}")

train_df.show(3)
test_df.show(3)

Training: 858320, test: 214887
+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 19| 1| 5.0|
 19| 3| 4.0|
 19| 7| 5.0|
+------+-------+------+
only showing top 3 rows

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 19| 6| 5.0|
 19| 25| 3.0|
 19| 55| 3.0|
+------+-------+------+
only showing top 3 rows

### (2b) Benchmark Model

Let's always predict the average movie rating in our dataset to use as our benchmark model, and see what our RMSE is.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import lit, avg

average_rating = train_df.select(avg("rating")).first()[0]

benchmark_df = train_df.withColumn("prediction", lit(average_rating))

reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")
baseline_rmse = reg_eval.evaluate(benchmark_df)

print(f"Baseline RMSE: {baseline_rmse:.3}")

Baseline RMSE: 1.05

In [0]:
benchmark_df.show(5)

+------+-------+------+-----------------+
userId|movieId|rating| prediction|
+------+-------+------+-----------------+
 19| 1| 5.0|3.515378879671917|
 19| 3| 4.0|3.515378879671917|
 19| 7| 5.0|3.515378879671917|
 19| 14| 4.0|3.515378879671917|
 19| 17| 4.0|3.515378879671917|
+------+-------+------+-----------------+
only showing top 5 rows

### (2c) Alternating Least Squares

In this part, we will use the Apache Spark ML Pipeline implementation of Alternating Least Squares, <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.recommendation.ALS.html?highlight=als#pyspark.ml.recommendation.ALS" target="_blank">ALS</a>. To determine the best values for the hyperparameters, we will use ALS to train several models, and then we will select the best model and use the parameters from that model in the rest of this lab exercise.

The process we will use for determining the best model is as follows:
1. Pick a set of model hyperparameters. The most important hyperparameter to model is the *rank*, which is the number of columns in the Users matrix or the number of rows in the Movies matrix. In general, a lower rank will mean higher error on the training dataset, but a high rank may lead to <a href="https://en.wikipedia.org/wiki/Overfitting" target="_blank">overfitting</a>.  We will train models with ranks of 4 and 12 using the **`train_df`** dataset.

2. Set the appropriate values:
    * The "User" column will be set to the values in our **`userId`** DataFrame column.
    * The "Item" column will be set to the values in our **`movieId`** DataFrame column.
    * The "Rating" column will be set to the values in our **`rating`** DataFrame column.
    * **`nonnegative`** = True (whether to use nonnegative constraint for least squares)
    * **`regParam`** = 0.1.
    
   **Note**: Read the documentation for the ALS class **carefully**. It will help you accomplish this step.

4. Create multiple models using the **`ParamGridBuilder`** and the **`CrossValidator`**, one for each of our rank values.

6. We'll keep the model with the lowest error rate. Such a model will be selected automatically by the CrossValidator.

In [0]:
# TODO
from pyspark.ml.recommendation import ALS

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", maxIter=5, seed=seed, coldStartStrategy="drop", regParam=0.1, nonnegative=True)

In [0]:
# Test our solution
assert als.getItemCol() == "movieId", f"Incorrect choice of {als.getItemCol()} for ALS item column."
assert als.getUserCol() == "userId", f"Incorrect choice of {als.getUserCol()} for ALS user column."
assert als.getRatingCol() == "rating", f"Incorrect choice of {als.getRatingCol()} for ALS rating column."

Now that we have initialized a model, we need to fit it to our training data, and evaluate how well it does on the validation dataset. Create a **`CrossValidator`** and **`ParamGridBuilder`** that will decide whether *rank* value *4* or *12* gives a lower *RMSE*.  

NOTE: This cell may take a few minutes to run.

In [0]:
# TODO
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import RegressionEvaluator

reg_eval = RegressionEvaluator(predictionCol='prediction',labelCol='rating',metricName='rmse') # Create RegressionEvaluator

grid = (ParamGridBuilder()
        .addGrid(als.rank,[4,12])
        .build())

seed = 42
cv = CrossValidator(numFolds=3, estimator=als, estimatorParamMaps=grid, evaluator=reg_eval, seed=seed) # Set number of folds to 3. Add grid, als, reg_eval, and seed           

cv_model = cv.fit(train_df)

my_model = cv_model.bestModel

print(f"The best model was trained with rank {my_model.rank}")

The best model was trained with rank 12

In [0]:
# Test our solution
assert my_model.rank == 12, f"Unexpected value for best rank. Expected 12, got {my_model.rank}"

### (2d) Testing Your Model

So far, we used the **`train_df`** dataset to select the best model. Since we used this dataset to determine what model is best, we cannot use it to test how good the model is; otherwise, we would be very vulnerable to <a href="https://en.wikipedia.org/wiki/Overfitting" target="_blank">overfitting</a>.  To decide how good our model is, we need to use the **`test_df`** dataset.  We will use the best model you created in part (2b) for predicting the ratings for the test dataset and then we will compute the RMSE.

The steps you should perform are:
* Run a prediction, using **`my_model`** as created above, on the test dataset (**`test_df`**), producing a new **`predicted_test_df`** DataFrame.
* Use the previously created RMSE evaluator, **`reg_eval`** to evaluate the filtered DataFrame.

In [0]:
# TODO

predicted_test_df = my_model.transform(test_df)

# Run the previously created RMSE evaluator, reg_eval, on the predicted_test_df DataFrame
test_rmse = reg_eval.evaluate(predicted_test_df)

print(f"The model had a RMSE on the test set of {test_rmse}")

The model had a RMSE on the test set of 0.8435883688660719

## Part 3: Predictions for Yourself
The ultimate goal of this lab exercise is to predict what movies to recommend to yourself.  In order to do that, you will first need to add ratings for yourself to the **`ratings_df`** dataset.

**(3a) Your Movie Ratings**

To help you provide ratings for yourself, we have included the following code to list the names and movie IDs of the 100 highest-rated movies that have at least 500 ratings.

In [0]:
movies_df.createOrReplaceTempView("movies")
ratings_df.createOrReplaceTempView("ratings")

In [0]:
%sql
SELECT movieId, title, AVG(rating) AS avg_rating, COUNT(*) AS num_ratings
FROM ratings r JOIN movies m ON (r.movieID = m.ID)
GROUP BY r.movieId, m.title
HAVING COUNT(*) > 500
ORDER BY avg_rating DESC
LIMIT 100

movieId,title,avg_rating,num_ratings
318,"Shawshank Redemption, The (1994)",4.441669159437631,3343
50,"Usual Suspects, The (1995)",4.339622641509434,2491
858,"Godfather, The (1972)",4.332802547770701,2198
527,Schindler's List (1993),4.329755178907721,2655
904,Rear Window (1954),4.2916194790486974,883
1221,"Godfather: Part II, The (1974)",4.290697674418604,1419
2019,Seven Samurai (Shichinin no samurai) (1954),4.272161741835148,643
750,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),4.248505550811273,1171
912,Casablanca (1942),4.24231950844854,1302
1198,Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),4.231166594920362,2323


The user ID 0 is unassigned, so we will use it for your ratings. We set the variable **`myUserId`** to 0 for you. 

Next, create a new DataFrame called **`my_ratings_df`**, with your ratings for at least 10 movie ratings. Each entry should be formatted as **`(myUserId, movieId, rating)`**.  As in the original dataset, ratings should be between 1 and 5 (inclusive). 

If you have not seen at least 10 of these movies, you can increase the parameter passed to **`LIMIT`** in the above cell until there are 10 movies that you have seen (or you can also guess what your rating would be for movies you have not seen).

In [0]:
# TODO
myUserId = 0

# Note that the movie IDs are the *last* number on each line. A common error was to use the number of ratings as the movie ID.
myRatedMovies = [
     (myUserId, 260,  5),
    (myUserId, 1196, 5),
    (myUserId, 480,  5),
    (myUserId, 1210, 5),
    (myUserId, 589,  5),
    (myUserId, 2028, 1),
    (myUserId, 1580, 5),
    (myUserId, 110,  1),
    (myUserId, 2571, 5),
    (myUserId, 1270, 5),
    (myUserId, 2716, 5),
    (myUserId, 2791, 1),
    (myUserId, 1265, 1)
     # The format of each line is (myUserId, movie ID, your rating)
     # For example, to give the movie "Star Wars: Episode IV - A New Hope (1977)" a five rating, you would add the following line:
     #   (myUserId, 260, 5),
]

my_ratings_df = spark.createDataFrame(myRatedMovies, ["userId", "movieId", "rating"])
display(my_ratings_df.limit(10))

userId,movieId,rating
0,260,5
0,1196,5
0,480,5
0,1210,5
0,589,5
0,2028,1
0,1580,5
0,110,1
0,2571,5
0,1270,5


### (3b) Add Your Movies to Training Dataset

Now that you have ratings for yourself, you need to add your ratings to the **`train_df`** dataset so that the model you train will incorporate your preferences.  Spark's <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.union.html?highlight=union#pyspark.sql.DataFrame.union" target="_blank">union()</a> transformation combines two DataFrames; use **`union()`** to create a new training dataset that includes your ratings and the data in the original training dataset.

In [0]:
# TODO
training_with_my_ratings_df = train_df.union(my_ratings_df)

count_diff = training_with_my_ratings_df.count() - train_df.count()
print(f"The training dataset now has {count_diff} more entries than the original training dataset")
assert (count_diff == my_ratings_df.count())

The training dataset now has 13 more entries than the original training dataset

### (3c) Train a Model with Your Ratings

Now, train a model with your ratings added and the parameters you used in in part (2b) and (2c). Make sure you include **all** of the parameters.

**Note**: This cell will take about 1 minute to run.

In [0]:
# TODO

als.setRank(12)

# Create the model with these parameters
my_ratings_model = als.fit(training_with_my_ratings_df)

### (3d) Predict Your Ratings

Now that we have trained a new model, let's predict what ratings you would give to the movies that you did not already provide ratings for. The code below filters out all of the movies you have rated, and creates a **`predicted_ratings_df`** DataFrame of the predicted ratings for all of your unseen movies.

In [0]:
# Create a list of the my rated movie IDs 
my_rated_movie_ids = [x[1] for x in myRatedMovies]

# Filter out the movies I already rated.
not_rated_df = movies_df.filter(~ movies_df["ID"].isin(my_rated_movie_ids))

# Rename the "ID" column to be "movieId", and add a column with myUserId as "userId".
my_unrated_movies_df = not_rated_df.withColumnRenamed("ID", "movieId").withColumn("userId", lit(myUserId))       

# Use my_ratings_model to predict ratings for the movies that I did not manually rate.
predicted_ratings_df = my_ratings_model.transform(my_unrated_movies_df)

### (3e) Predict Your Ratings

We have our predicted ratings. Now we can print out the 25 movies with the highest predicted ratings.

In [0]:
predicted_ratings_df.createOrReplaceTempView("predictions")

Let's take a look at the raw predictions:

In [0]:
%sql 
SELECT * FROM predictions

movieId,title,userId,prediction
1,Toy Story (1995),0,3.5115123
2,Jumanji (1995),0,3.2911236
3,Grumpier Old Men (1995),0,2.6562324
4,Waiting to Exhale (1995),0,2.6172197
5,Father of the Bride Part II (1995),0,2.8033192
6,Heat (1995),0,3.1068442
7,Sabrina (1995),0,3.0836391
8,Tom and Huck (1995),0,2.9490232
9,Sudden Death (1995),0,2.286385
10,GoldenEye (1995),0,3.3851519


Now print out the 25 movies with the highest predicted ratings. We will only include movies that have at least 75 ratings in total.

In [0]:
%sql
SELECT p.title, p.prediction AS your_predicted_rating
FROM ratings r INNER JOIN predictions p 
ON (r.movieID = p.movieID)
WHERE p.userId = 0
GROUP BY p.title, p.prediction
HAVING COUNT(*) > 75
ORDER BY p.prediction DESC
LIMIT 25

title,your_predicted_rating
"Avengers, The (2012)",4.4470096
Hellboy II: The Golden Army (2008),4.291554
"Dark Knight, The (2008)",4.2507596
Guardians of the Galaxy (2014),4.2483273
"Amazing Spider-Man, The (2012)",4.1477776
Dr. Horrible's Sing-Along Blog (2008),4.1444936
"Dark Knight Rises, The (2012)",4.128508
Clue (1985),4.117393
Star Trek (2009),4.1078267
"Bourne Identity, The (1988)",4.102108


-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>